---
## Setup Security

In [ ]:
-- USE ROLE SECURITYADMIN;

-- GRANT USAGE ON DATABASE HISTORICAL_QUOTES_DEMO TO ROLE OPENFLOW_RUNTIME1_ROLE;
-- GRANT USAGE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE OPENFLOW_RUNTIME1_ROLE;
-- GRANT CREATE TABLE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE OPENFLOW_RUNTIME1_ROLE;

-- GRANT USAGE ON DATABASE HISTORICAL_QUOTES_DEMO TO ROLE ACCOUNTADMIN;
-- GRANT USAGE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE ACCOUNTADMIN;
-- GRANT SELECT ON ALL TABLES IN SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC to ROLE ACCOUNTADMIN;
-- GRANT CREATE TABLE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE ACCOUNTADMIN;
-- GRANT CREATE DYNAMIC TABLE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE ACCOUNTADMIN;

-- GRANT CREATE STREAMLIT ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE PUBLIC;
-- GRANT CREATE STAGE ON SCHEMA HISTORICAL_QUOTES_DEMO.PUBLIC TO ROLE PUBLIC;
-- GRANT USAGE ON WAREHOUSE "SYSTEM$STREAMLIT_NOTEBOOK_WH" TO ROLE PUBLIC;

In [ ]:
-- USE ROLE ACCOUNTADMIN;

-- ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_EU';

-- GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE ACCOUNTADMIN;
-- CREATE DATABASE IF NOT EXISTS snowflake_intelligence;
-- CREATE SCHEMA IF NOT EXISTS snowflake_intelligence.agents;
-- GRANT USAGE ON DATABASE snowflake_intelligence TO ROLE ACCOUNTADMIN;
-- GRANT USAGE ON SCHEMA snowflake_intelligence.agents TO ROLE ACCOUNTADMIN;
-- GRANT CREATE AGENT ON SCHEMA snowflake_intelligence.agents TO ROLE ACCOUNTADMIN;

---
## Create Dynamic Table
`HISTORICAL_QUOTES_TYPED` is a copy of the table created by the Openflow Kafka connector, but with correct types for date and currency columns.

In [ ]:
SELECT
    TABLE_NAME,
    TABLE_TYPE,
    TABLE_OWNER
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'PUBLIC'
AND TABLE_NAME LIKE '%-STOCK-QUOTES';

In [ ]:
WITH typed AS (
    SELECT 
        SYMBOL,
        TO_DATE(DATE) AS QUOTE_DATE,
        TO_DOUBLE(LTRIM(CLOSE_LAST, '$')) AS CLOSE_LAST_USD,
        VOLUME,
        TO_DOUBLE(LTRIM(OPEN, '$')) AS OPEN_USD,
        TO_DOUBLE(LTRIM(HIGH, '$')) AS HIGH_USD,
        TO_DOUBLE(LTRIM(LOW, '$')) AS LOW_USD
    FROM "HISTORICAL-STOCK-QUOTES"
)
SELECT
    SYMBOL,
    QUOTE_DATE,
    CLOSE_LAST_USD,
    VOLUME
FROM typed
ORDER BY SYMBOL, QUOTE_DATE DESC
LIMIT 10;

In [ ]:
-- USE ROLE ACCOUNTADMIN;
-- USE DATABASE HISTORICAL_QUOTES_DEMO;

-- CREATE OR REPLACE DYNAMIC TABLE "HISTORICAL_QUOTES_TYPED"
--     TARGET_LAG = '1 hour'
--     WAREHOUSE = 'COMPUTE_WH'
--     AS
--         SELECT 
--             SYMBOL,
--             TO_DATE(DATE) AS QUOTE_DATE,
--             TO_DOUBLE(LTRIM(CLOSE_LAST, '$')) AS CLOSE_LAST_USD,
--             VOLUME,
--             TO_DOUBLE(LTRIM(OPEN, '$')) AS OPEN_USD,
--             TO_DOUBLE(LTRIM(HIGH, '$')) AS HIGH_USD,
--             TO_DOUBLE(LTRIM(LOW, '$')) AS LOW_USD
--         FROM "HISTORICAL-STOCK-QUOTES";

In [ ]:
------------------------------
-- Speed things up a bit
------------------------------
ALTER DYNAMIC TABLE "HISTORICAL_QUOTES_TYPED" SET TARGET_LAG = '1 minute';
-- ALTER DYNAMIC TABLE "HISTORICAL_QUOTES_TYPED" SET TARGET_LAG = '1 hour';

In [ ]:
SELECT 
    SYMBOL,
    COUNT(QUOTE_DATE) AS num_quotes,
    MAX(VOLUME),
    MAX(CLOSE_LAST_USD),
    MAX(QUOTE_DATE)
FROM "HISTORICAL_QUOTES_TYPED"
GROUP BY SYMBOL
ORDER BY SYMBOL;

---
## Stock Price Forecast

In [ ]:
-- CREATE OR REPLACE VIEW "TIMESERIES-STOCK-QUOTES" AS 
--     SELECT 
--         SYMBOL,
--         TO_DATE(DATE) AS QUOTE_DATE,
--         TO_DOUBLE(LTRIM(CLOSE_LAST, '$')) AS CLOSE_LAST_USD        
--     FROM "HISTORICAL-STOCK-QUOTES"
--     ORDER BY SYMBOL, QUOTE_DATE;

SELECT * 
FROM "TIMESERIES-STOCK-QUOTES"
WHERE SYMBOL = 'TSLA'
ORDER BY SYMBOL, QUOTE_DATE DESC
LIMIT 10;

In [ ]:
------------------------------
-- Create forecast model 
------------------------------
-- CREATE OR REPLACE SNOWFLAKE.ML.FORECAST stock_quote_forecast(
--   INPUT_DATA => TABLE("TIMESERIES-STOCK-QUOTES"),
--   SERIES_COLNAME => 'SYMBOL',
--   TIMESTAMP_COLNAME => 'QUOTE_DATE',
--   TARGET_COLNAME => 'CLOSE_LAST_USD'
-- );

SHOW SNOWFLAKE.ML.FORECAST;
-- DROP SNOWFLAKE.ML.FORECAST stock_quote_forecast;

In [ ]:
------------------------------
-- Create table with forecasts for next 3 months
-- 5 working days a week, 4 weeks a month, 3 months = 60 periods
------------------------------

-- CREATE OR REPLACE TABLE "FORECAST-STOCK-QUOTES" AS
--   SELECT * 
--   FROM TABLE(stock_quote_forecast!FORECAST(FORECASTING_PERIODS => 60));

SELECT 
    SERIES,
    COUNT(TS) AS NUM_FORECASTS
FROM "FORECAST-STOCK-QUOTES"
GROUP BY SERIES
ORDER BY SERIES;

In [ ]:
SELECT 
    SERIES,
    TS,
    FORECAST,
    LOWER_BOUND,
    UPPER_BOUND
FROM "FORECAST-STOCK-QUOTES"
WHERE SERIES = 'TSLA' AND TS > CURRENT_TIMESTAMP()
ORDER BY SERIES, TS
LIMIT 10;

---
## Cortex Search for Earnings Reports

In [ ]:
-- CREATE OR REPLACE STAGE EARNINGS_REPORTS_STAGE
--     ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE')
--     DIRECTORY = ( ENABLE = TRUE );

-- GRANT READ ON STAGE EARNINGS_REPORTS_STAGE TO ROLE ACCOUNTADMIN;
-- GRANT WRITE ON STAGE EARNINGS_REPORTS_STAGE TO ROLE ACCOUNTADMIN;

-- ALTER STAGE EARNINGS_REPORTS_STAGE REFRESH;

SELECT 
    COUNT(*) as total_documents,
    COUNT(DISTINCT RELATIVE_PATH) as unique_files,
    ROUND(AVG(SIZE)/1024/1024, 2) as avg_size_mb
FROM DIRECTORY(@EARNINGS_REPORTS_STAGE);

-- REMOVE @EARNINGS_REPORTS_STAGE;

In [ ]:
----------------------------------------
-- Create table to hold the extracted text from the PDF files
--
-- See AISQL AI_PARSE_DOCUMENT
--   https://docs.snowflake.com/en/user-guide/snowflake-cortex/parse-document
----------------------------------------

-- USE ROLE ACCOUNTADMIN;
-- USE DATABASE HISTORICAL_QUOTES_DEMO;

-- CREATE OR REPLACE TABLE EARNINGS_REPORTS_PARSED (
--   relative_path VARCHAR(), 
--   markdown VARIANT
-- );

-- INSERT INTO "EARNINGS_REPORTS_PARSED" (relative_path, markdown)
-- WITH staged_reports AS (
--     SELECT
--         relative_path
--     FROM DIRECTORY(@EARNINGS_REPORTS_STAGE)
-- )
-- SELECT
--     relative_path,
--     AI_PARSE_DOCUMENT (
--         TO_FILE('@EARNINGS_REPORTS_STAGE', relative_path),
--         {'mode': 'LAYOUT'}
--     ) AS markdown
-- FROM staged_reports;

SELECT
    relative_path,
    markdown
FROM "EARNINGS_REPORTS_PARSED"
WHERE relative_path LIKE 'TSLA_%'
ORDER BY relative_path
LIMIT 10;

In [ ]:
----------------------------------------
-- Chunk the text based on paragraph separators
--
-- See SPLIT_TEXT_RECURSIVE_CHARACTER
--   https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex
----------------------------------------

-- USE ROLE ACCOUNTADMIN;
-- USE DATABASE HISTORICAL_QUOTES_DEMO;

-- CREATE OR REPLACE TABLE EARNINGS_REPORTS_CHUNKS (
--   relative_path VARCHAR(), 
--   chunk STRING
-- );

-- INSERT INTO "EARNINGS_REPORTS_CHUNKS" (relative_path, chunk)
-- WITH report_chunks AS (
--     SELECT
--         relative_path,
--         SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
--             markdown:content::STRING, -- Extract the 'content' field from the JSON
--             'markdown',               -- Format type
--             2000,                     -- Chunk size (in tokens)
--             100,                      -- Overlap size
--             ['\n\n']                  -- Paragraph separators
--         ) AS chunks
--     FROM "EARNINGS_REPORTS_PARSED"
-- )
-- SELECT
--     relative_path,
--     c.value AS chunk -- Extract each chunk of the parsed text
-- FROM report_chunks,
-- LATERAL FLATTEN(INPUT => chunks) c;

SELECT
    relative_path,
    chunk
FROM "EARNINGS_REPORTS_CHUNKS"
WHERE relative_path LIKE 'TSLA_%'
ORDER BY relative_path
LIMIT 10;

In [ ]:
-- USE ROLE ACCOUNTADMIN;
-- USE DATABASE HISTORICAL_QUOTES_DEMO;

-- CREATE OR REPLACE CORTEX SEARCH SERVICE EARNINGS_REPORTS_SEARCH
-- ON chunk
-- WAREHOUSE = 'COMPUTE_WH'
-- TARGET_LAG = '5 minutes'
-- AS (
--     SELECT
--         relative_path,
--         chunk 
--     FROM EARNINGS_REPORTS_CHUNKS
-- );

-- ALTER CORTEX SEARCH SERVICE EARNINGS_REPORTS_SEARCH 
-- SET TARGET_LAG = '15 minutes';

In [ ]:
-- SHOW CORTEX SEARCH SERVICES LIKE 'EARNINGS_REPORTS_SEARCH';

SELECT *
FROM TABLE(
    CORTEX_SEARCH_DATA_SCAN(
        SERVICE_NAME => 'EARNINGS_REPORTS_SEARCH'
    )
)
WHERE relative_path LIKE 'TSLA_%'
LIMIT 10;

---
## Cleanup After Yourself!

In [ ]:
----------------------------------------
-- Don't forget to drop the dynamic table to avoid wasting dollars
----------------------------------------
-- DROP DYNAMIC TABLE "HISTORICAL_QUOTES_TYPED";

In [ ]:
----------------------------------------
-- Truncate the Kafka table
----------------------------------------
-- TRUNCATE TABLE IF EXISTS "HISTORICAL-STOCK-QUOTES";

In [ ]:
----------------------------------------
-- Drop earnings reports staging table
----------------------------------------
-- DROP STAGE EARNINGS_REPORTS_STAGE;

In [ ]:
----------------------------------------
-- Drop the stock price forecast table and model
----------------------------------------
-- DROP TABLE "FORECAST-STOCK-QUOTES";
-- DROP SNOWFLAKE.ML.FORECAST stock_quote_forecast;